**Classificação - Mineração de Dados**


**Nome: Davi Augusto Neves Leite**

**Data de Entrega: 10/10/2023**


---


# **Materiais**


Os principais recursos para a execução desta atividade podem ser vistos a seguir.

1. **Software**

- Sistemas Operacionais: Windows 11 para _desktop_;
- Ambiente de Desenvolvimento Integrado: Microsoft Visual Studio Code;
- Linguagem de Programação: Python 3.12.0 64-bit.

2. **Hardware**

- Notebook pessoal Lenovo Ideapad 330-15IKB com: processador Intel Core i7-8550U, HDD WD Blue WD10SPZX de 1TB, SSD Crucial BX500 de 1TB, 12 GB DDR4 de Memória RAM e placa de vídeo NVIDIA GeForce MX150 (2 GB GDDR5 de memória).


---


# **Instalação das Bibliotecas Principais**

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a instalação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install plotly
%pip install scikit-learn


---


# **Importação das Bibliotecas Principais**

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a importação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [ ]:
import numpy as np  # Manipulação de listas
import pandas as pd  # Manipulação de tabelas
import seaborn as sbn  # Geração de gráficos estatísticos
import plotly.express as px  # Outro para geração de gráficos
import matplotlib.pyplot as plt  # Geração de gráficos de listas
import sklearn as skl  # Biblioteca para pré-processamento
from copy import copy as cp  # Possibilitar copiar os objetos

# Ignorar os avisos não importantes durante a execução deste notebook
import warnings

warnings.filterwarnings("ignore")

---


# **Conjunto de Dados: _Fashion MNIST_**


**Descrição do Dataset:** este conjunto é composto por **70 mil imagens** a respeito de **10 peças de roupas distintas**. Cada imagem possui a **resolução de 28x28 (784 pixels) em escala de cinza e 256 níveis de cinza**. Desse total, exitem **7000 imagens** para cada uma das peças de roupas, ou classes.

Cada classe pode ser vista a seguir.

1. Camiseta
2. Calça
3. Pulôver
4. Vestido
5. Casaco
6. Sandália
7. Camisa
8. Tênis
9. Bolsa
10. Bota de Tornozelo

Este conjunto de dados pode ser acessado por meio de: [Fashion MNIST](https://www.openml.org/search?type=data&status=active&id=40996)
(última data de acesso: 02 de out. de 2023).


**Para fins práticos, apenas as classes _camiseta_, _vestido_, _casaco_ e _camisa_ serão utilizadas para esta atividade, tendo em vista que, por se tratar de uma base de dados de imagens, o processamento necessário para classificação é elevado.**


## **Importação da Base de Dados**


Para importar as imagens, utiliza-se o módulo **scikit-learn** capaz de carregar diversas bases de dados através do portal **OpenML**, o qual essa base de dados está disponível online.


In [ ]:
# Importação da função necessária para importação de base de dados OpenML
from sklearn.datasets import fetch_openml

# Carrega as imagens e suas classes separadamente
images, targets = fetch_openml(
    "Fashion-MNIST", return_X_y=True, as_frame=True, parser="auto"
)

# Conversão das imagens para NumPy
images = images.to_numpy()
targets = targets.to_numpy().astype(int)

## **Pré-Processamento dos Dados**


### Separação das Classes de Interesse

Inicia-se com a separação das classes de interesse, isto é: _camiseta_, _vestido_, _casaco_ e _camisa_. Cada qual será referida, respectivamente, pela identificação de 0 a 3 (_0 - camiseta_, _1 - vestido_, ...).


In [ ]:
# Definição das classes de interesse
labels_class = {0: 0, 3: 1, 4: 2, 6: 3}

# Variáveis de seleção das imagens e classes respectivas
selected_images = []
selected_targets = []

# Criação de um dicionário com os rótulos das classes para fácil acesso
labels_desc = {0: "camiseta", 1: "vestido", 2: "casaco", 3: "camisa"}
labels_names = ["Camiseta", "Vestido", "Casaco", "Camisa"]

# Percorre as imagens da base de dados original
for idx, label in enumerate(targets):
    # Para cada rótulo de interesse, salvar as imagens e os rótulos
    if label in labels_class.keys():
        selected_images.append(images[idx])
        selected_targets.append(labels_class[targets[idx]])

# Conversão para NumPy
selected_images = np.array(selected_images)
selected_targets = np.array(selected_targets)

print("Imagens Selecionadas:\n", selected_images)
print("\nRótulos Selecionados: ", selected_targets)
print("\nTotal de Dados: ", selected_images.shape[0])
print("\nCaracterísticas Totais: ", selected_images.shape[1])

### Exibição das Classes de Interesse

Abaixo, é possível visualizar uma amostra de cada classe, por meio do **matplotlib**.


In [ ]:
# Selecionando um índice de cada classe
idx_0 = np.where(selected_targets == 0)[0][0]
idx_1 = np.where(selected_targets == 1)[0][0]
idx_2 = np.where(selected_targets == 2)[0][0]
idx_3 = np.where(selected_targets == 3)[0][0]
idx_example_images = np.array([idx_0, idx_1, idx_2, idx_3])

# Definindo o tamanho da figura
plt.figure(figsize=(10, 8))

# Definindo o número de linhas e colunas das subfiguras
fig_n_rows = 1
fig_n_cols = 5

# Mostrando as amostras de cada classe
for label, image_idx in enumerate(labels_desc.keys()):
    plt.subplot(fig_n_rows, fig_n_cols, label + 1)
    plt.title(f'Classe "{labels_desc[label]}"')
    plt.imshow(
        selected_images[idx_example_images[image_idx]].reshape(28, 28), cmap="gray"
    )
    plt.axis("off")
plt.show()

### Tratamento de Dados Perdidos ou Inexistentes (NaN)


Para verificar se algum dado está faltando, **caso não seja indicado pela descrição do _dataset_**, pode ser realizado a seguinte operação de força-bruta:


In [ ]:
# Verificando o número de dados faltantes a partir do NumPy
missing_selected_image = np.isnan(selected_images)
missing_selected_image = np.sum(missing_selected_image)
print("Número de Dados Perdidos: {0}".format(missing_selected_image))

Como é possível ver, não há nenhum dado perdido neste _dataset_ e, desta forma, não é necessário realizar nenhum método de tratamento neste contexto.


### Normalização dos Dados


Para normalizar os dados via _Standardization (Z-Score)_ deste **dataset**, basta aplicar as seguintes operações:


In [ ]:
# Função responsável pela normalização via Z-Score
from sklearn.preprocessing import StandardScaler

# Mostrando os dados não normalizados
print("Dados Não Normalizados")
print(
    "\tMédia: {0} | Desvio-Padrão: {1}".format(
        np.mean(selected_images), np.std(selected_images)
    )
)
print(selected_images)


# Aplicando a Normalização com Z-Score
selected_images = StandardScaler().fit_transform(selected_images)
print("\n")

# Mostrando os dados normalizados
print("Dados Normalizados com Z-Score")
print(
    "\tMédia: {0} | Desvio-Padrão: {1}".format(
        np.mean(selected_images), np.std(selected_images)
    )
)
print(selected_images)

### Redução de Dimensionalidade: _Principal Component Analysis_


Para reduzir a dimensionalidade deste **dataset**, é recomendado o uso do _Principal Component Analysis_ (PCA). Desta forma:


In [ ]:
# Importação da função do PCA do sklearn
from sklearn.decomposition import PCA

# Definindo o número de componentes do PCA
n_components = 0.9

# Aplicando o PCA
pca = PCA(n_components=n_components, copy=True, whiten=False)
projected_data = pca.fit_transform(selected_images)

# Mostrando os dados projetados com PCA
print("Dados Projetados com PCA")
print(projected_data)
print("\n")

# Segundo: mostrando a matriz de covariância do PCA
print("Variâncias")
print(pca.explained_variance_ratio_)
print("\n")

# Transformando o conjunto de dados em DataFrame para melhor manipulação
column_name = ["pixel" + str(i) for i in range(1, 785)]
selected_images_zscore_frame = pd.DataFrame(selected_images, columns=column_name)

# Terceiro: mostrando os componentes do PCA
component_names = ["component {}".format(i) for i in range(len(pca.components_))]
components_pca = pd.DataFrame(
    data=pca.components_,
    index=component_names,
    columns=selected_images_zscore_frame.columns,
)
components_pca.head()

É possível, por exemplo, visualizar a variação do PCA na medida em que se aumentam as características (ou dimensões). A chamada **variância explicada** exprime exatamente a ideia de quanta variabilidade é possível capturar do conjunto de dados, sendo possível, a partir desta análise, levar a um melhor desempenho de treinamento do modelo.


In [ ]:
import plotly.graph_objects as px_go

# Gráfico da variância do PCA, a partir dos componentes
# exp_var_cum = np.cumsum(pca.explained_variance_ratio_)
exp_var_cum = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=3) * 100)

fig_pca = px_go.Figure(
    data=px_go.Scatter(x=list(range(1, len(exp_var_cum) + 1)), y=exp_var_cum)
)
fig_pca.update_layout(
    title="Variância Explicada do PCA",
    xaxis_title="# de Características",
    yaxis_title="% Variância Explicada",
)

---


## **Classificação**


Uma das características dos _datasets MNIST_ é de terem pixels bem diferenciados entre si e, portanto, é possível aplicar os classificadores sem a necessidade de uma extração prévia de características, como por meio das técnicas de extração de texturas como os **Descritores de Haralick** ou do **_Local Binary Pattern_**.

Para validar os dados, será utilizado a técnica de **_cross-validation_**. Nesta técnica, os dados são separados entre um conjunto de treinamento e um de teste. O de treinamento, como o nome sugere, serve para o classificador realizar predições e "aprender" as diferentes classes. Já o de teste é utilizado para verificar o quanto o classificador "pode aprender" com o treinamento, ou seja, são feitas predições das classes neste conjunto.

Para isso, é realizado a separação de 80% do conjunto total para treinamento e 20% para conjunto de teste.


In [ ]:
# Função para separação dos conjuntos
from sklearn.model_selection import train_test_split

# Divisão dos conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(
    selected_images, selected_targets, test_size=0.2, random_state=42
)

print("Conjunto de Treinamento\n", X_train)
print("Rótulos de Treinamento\n", y_train)
print("Quantidade de Dados de Treinamento: ", X_train.shape[0])
print()
print("Conjunto de Teste\n", X_test)
print("Rótulos de Teste\n", y_test)
print("Quantidade de Dados de Teste: ", X_test.shape[0])

### _Decision Trees_


As Árvores de Decisão são modelos de aprendizado de máquina supervisionado que representam decisões e suas consequências diretas, em forma de árvore. Neste modelo, cada nó interno representa um teste em um atributo, cada ramo representa um resultado do teste e cada nó folha representa uma classe. Por conta disso, as Árvores de Decisão são altamente interpretáveis, capazes de lidar com dados categóricos e numéricos, ainda que possam ser propensas ao _overfitting_ se não ajustadas corretamente.

Desta forma, para aplicar a Árvore de Decisão para o _dataset_ proposto basta realizar as seguintes etapas a seguir.


In [ ]:
# Métricas para análise de desempenho
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score,
)

# Árvore de Decisão para Classificação
from sklearn.tree import DecisionTreeClassifier

# Definindo hiperparâmetros
max_depth = 100

# Criando a estrutura básica
dec_tree = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

# Realizando o treinamento
dec_tree.fit(X=X_train, y=y_train)

# Realizando a predição no conjunto de teste
y_pred = dec_tree.predict(X=X_test)

# Mostrando a acurácia obtida
dec_tree_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {dec_tree_accuracy:.2f}")

Como é possível visualizar, a Árvore de Decisão com **100 nós de profundidade máxima** conseguiu obter **76% de acurácia para classificação do conjunto de teste**. Além da acurácia, é possível visualizar o relatório completo abaixo.


In [ ]:
# Mostrando o relatório completo de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=labels_names))

Outra importante métrica é a matriz de confusão. Essa métrica nada mais é do que uma tabela que compara as previsões do modelo com os valores reais dos dados. Nela, são mostrados quantos exemplos de cada classe foram classificados corretamente (verdadeiros positivos e verdadeiros negativos) e quantos foram classificados de forma incorreta (falsos positivos e falsos negativos).

A seguir, é possível visualizar a matriz de confusão e o respectivo mapa de calor.


In [ ]:
# Criando e mostrando a matriz de confusão com relação ao conj. de teste
print("Matriz de Confusão:\n")
print(confusion_matrix(y_test, y_pred, labels=dec_tree.classes_))
dec_tree_cm_disp = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=labels_names, cmap="Blues"
).figure_.suptitle("Matriz de Confusão")

### _Bagging_


O _Bagging_ consiste em uma técnica de _ensemble learning_ em que visa melhorar a precisão e reduzir a ocorrência de _overfitting_ de determinado modelo. Para tanto, nesta técnica são construídos vários modelos independentes, utilizando subconjuntos aleatórios do conjunto original (com reposição) e, após isso, é agregado suas previsões para a tomada de decisão final. Em outras palavras e tomando como exemplo a Árvore de Decisão, seriam aplicados várias Árvores de Decisão independente para subconjuntos do conjunto de treinamento e, a partir do resultado final, haveria a combinação de todos os modelos. No caso da classificação, a classe mais frequente entre as previsões obtidas das Árvores de Decisão seria a escolhida no final do processo.

Desta forma, para aplicar o _Bagging com Árvore de Decisão_ para o _dataset_ proposto basta realizar as seguintes etapas a seguir.


In [ ]:
# Métricas para análise de desempenho
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score,
)

# Bagging para Classificação juntamente com a Árvore de Decisão
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Definindo hiperparâmetros do Bagging
max_samples = 0.6
max_features = 0.6

# Definindo hiperparâmetros da Árvore de Decisão
max_depth = 100

# Criando a estrutura básica
bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=max_depth, random_state=42),
    max_samples=max_samples,
    max_features=max_features,
    random_state=42,
)

# Realizando o treinamento
bagging.fit(X=X_train, y=y_train)

# Realizando a predição no conjunto de teste
y_pred = bagging.predict(X=X_test)

# Mostrando a acurácia obtida
bagging_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {bagging_accuracy:.2f}")

Como é possível visualizar, o _Bagging com Árvore de Decisão_, utilizando **60% de amostras máximas** e **60% de características máximas**, conseguiu obter **82% de acurácia para classificação do conjunto de teste**. Além da acurácia, é possível visualizar o relatório completo abaixo.


In [ ]:
# Mostrando o relatório completo de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=labels_names))

A seguir, é possível visualizar a matriz de confusão e o respectivo mapa de calor.


In [ ]:
# Criando e mostrando a matriz de confusão com relação ao conj. de teste
print("Matriz de Confusão:\n")
print(confusion_matrix(y_test, y_pred, labels=bagging.classes_))
bagging_cm_disp = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=labels_names, cmap="Blues"
).figure_.suptitle("Matriz de Confusão")

### _AdaBoost_


Diferentemente do _Bagging_, o _Boosting_ é uma técnica de _ensemble learning_ que combina múltiplos modelos fracos para a criação de um modelo forte. Neste caso, os modelos são treinados sequencialmente e, desta forma, cada modelo tenta corrigir os erros do modelo anterior. A ideia é de que as classes "mal classificadas" recebam um peso maior a cada iteração e modelo. O _Adaptative Boosting_, ou _AdaBoost_, é um algoritmo de _Boosting_ que é particularmente eficaz em problemas de classificação binária, se destacando nas classes de "difícil previsão" de modelos normais.

Desta forma, para aplicar o _AdaBoost com Árvore de Decisão_ para o _dataset_ proposto basta realizar as seguintes etapas a seguir.


In [ ]:
# Métricas para análise de desempenho
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score,
)

# AdaBoost para Classificação, com Árvore de Decisão
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Definindo hiperparâmetros do AdaBoost
n_estimators = 100

# Definindo hiperparâmetros da Árvore de Decisão
max_depth = 100

# Criando a estrutura básica
ada_boost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=max_depth, random_state=42),
    n_estimators=n_estimators,
    random_state=42,
)

# Realizando o treinamento
ada_boost.fit(X=X_train, y=y_train)

# Realizando a predição no conjunto de teste
y_pred = ada_boost.predict(X=X_test)

# Mostrando a acurácia obtida
ada_boost_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {ada_boost_accuracy:.2f}")

Como é possível visualizar, o _AdaBoost com Árvore de Decisão_ utilizando **100 estimadores** conseguiu obter **77% de acurácia para classificação do conjunto de teste**. Além da acurácia, é possível visualizar o relatório completo abaixo.


In [ ]:
# Mostrando o relatório completo de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=labels_names))

A seguir, é possível visualizar a matriz de confusão e o respectivo mapa de calor.


In [ ]:
# Criando e mostrando a matriz de confusão com relação ao conj. de teste
print("Matriz de Confusão:\n")
print(confusion_matrix(y_test, y_pred, labels=ada_boost.classes_))
ada_boost_cm_disp = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=labels_names, cmap="Blues"
).figure_.suptitle("Matriz de Confusão")

### _Random Forest_


O _Random Forest_ é um algoritmo de _ensemble learning_ que combina **múltiplas Árvores de Decisão** para melhorar a tomada de decisões. Nesta técnica, cada Árvore de Decisão é treinada em diferentes subconjuntos do conjunto de dados original. A principal diferença desta para com o uso de _Bagging com Árvore de Decisão_ está em que o _Random Forest_, durante o treinamento, seleciona aleatoriamente os subconjuntos de dados para cada divisão de nó. Em outras palavras, enquanto que no _Bagging_ para cada divisão em um nó da árvore **todos** os atributos são considerados, no _Random Forest_ para cada divisão de nó é levado em consideração diferentes conjuntos de atributos. Por conta disso, o _Random Forest_ reduz ainda mais a correlação entre as Árvores de Decisão, geralmente levando a modelos mais robustos e precisos.

Desta forma, para aplicar o _Random Forest_ para o _dataset_ proposto basta realizar as seguintes etapas a seguir.


In [ ]:
# Métricas para análise de desempenho
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score,
)

# Random Forest para Classificação
from sklearn.ensemble import RandomForestClassifier

# Definindo hiperparâmetros do Random Forest
n_estimators = 100
max_depth = 100

# Criando a estrutura básica
rand_forest = RandomForestClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=42
)

# Realizando o treinamento
rand_forest.fit(X=X_train, y=y_train)

# Realizando a predição no conjunto de teste
y_pred = rand_forest.predict(X=X_test)

# Mostrando a acurácia obtida
rand_forest_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {rand_forest_accuracy:.2f}")

Como é possível visualizar, o _Random Forest_ com **100 estimadores e 100 de profundidade máxima** conseguiu obter **85% de acurácia para classificação do conjunto de teste**. Além da acurácia, é possível visualizar o relatório completo abaixo.


In [ ]:
# Mostrando o relatório completo de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=labels_names))

A seguir, é possível visualizar a matriz de confusão e o respectivo mapa de calor.


In [ ]:
# Criando e mostrando a matriz de confusão com relação ao conj. de teste
print("Matriz de Confusão:\n")
print(confusion_matrix(y_test, y_pred, labels=rand_forest.classes_))
rand_forest_cm_disp = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=labels_names, cmap="Blues"
).figure_.suptitle("Matriz de Confusão")

### Ajuste de Hiperparâmetros do Melhor Modelo: **_Random Forest_**


In [ ]:
# Criando o gráfico de comparativo de acurácia entre modelos
acc_names_fig = np.array(
    ["Decision Tree (DT)", "Bagging (with DT)", "AdaBoost (with DT)", "Random Forest"]
)
acc_values_fig = np.array(
    [
        dec_tree_accuracy * 100,
        bagging_accuracy * 100,
        ada_boost_accuracy * 100,
        rand_forest_accuracy * 100,
    ]
)

plt.figure(figsize=(8, 5))
plt.bar(acc_names_fig, acc_values_fig)
plt.suptitle("Acurácia dos Modelos para Fashion-MNIST*")
plt.title("*: apenas quatro classes")
plt.xlabel("Modelos")
plt.ylabel("Acurácia Obtida (em %)")
plt.show()

Tendo em vista os resultados de **acurácia** obtidos por cada modelo executado anteriormente, e os quais são possíveis de visualizar no gráfico anterior, obteve-se que o melhor modelo foi o **Random Forest** com **85% de acurácia**. Desta forma, torna-se necessário verificar se é possível melhorar os hiperparâmetros para a obtenção de melhores resultados.

Para tanto, é possível aplicar a técnica de **_Grid Search_**, a qual consiste em realizar testes exaustivos com base em um conjunto de valores especificados para os hiperparâmetros. Em outras palavras, esta técnica testa todas as combinações de hiperparâmetros especificadas pelo usuário para um determinado modelo, obtendo a melhor combinação ao final dos testes.


In [ ]:
# Importação do Grid Search para melhorar o modelo
from sklearn.model_selection import GridSearchCV

# Criação de um dicionário para conter um conjunto de valores de teste
# Quatro números de estimadores máximo e quatro números de profundidade máxima
param_grid = [
    {
        "n_estimators": [10, 50, 100, 1000],
        "max_depth": [10, 50, 100, 1000],
    }
]

# Aplicação do Grid Search: execução exaustiva!
grid_search_rand_forest = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), param_grid=param_grid
)

# Treinamento do Grid Search
grid_search_rand_forest.fit(X=X_train, y=y_train)

# Mostrando os melhores parâmetros obtidos
print(f"Melhores hiperparâmetros: {grid_search_rand_forest.best_params_}")

# Realizando a predição no conjunto de teste
y_pred = grid_search_rand_forest.predict(X=X_test)

# Mostrando a acurácia obtida
grid_search_rand_forest_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {grid_search_rand_forest_accuracy:.2f}")

# Mostrando o relatório completo de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=labels_names))

Como é possível visualizar, o _Random Forest_ com **XX estimadores e XX de profundidade máxima** conseguiu obter **XX% de acurácia para classificação do conjunto de teste**. Também, é possível visualizar a matriz de confusão a seguir.


In [ ]:
# Criando e mostrando a matriz de confusão com relação ao conj. de teste
print("Matriz de Confusão:\n")
print(
    confusion_matrix(y_test, y_pred, labels=grid_search_rand_forest_accuracy.classes_)
)
rand_forest_cm_disp = ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=labels_names, cmap="Blues"
).figure_.suptitle("Matriz de Confusão")

---


## **Multiclasse para Binário**


### _One vs All (OVA) / One vs Rest (OVR)_


### _One vs One (OVO)_


---
